# ML Final Project

In this final project, I will analyze a dataset containing both real and fake job postings. The goal is to accurately classify a job posting based on its features. There are 17,880 observations. The distribution of the classes is not equal. There are 17,014 real job postings and 866 fake postings. The dataset has 18 features including one unique ID (job_id). The descriptions of the features are below.
 
| Columns	| Description |
| :-----------: | :-----------: |
| job_id	| Unique Job ID |
| title	| The title of the job posting entry. |
| location	| Geographical location of the job posting. |
| department	| Corporate department (e.g., sales). |
| salary_range	| Indicative salary range (e.g., $ 50,000-60,000). |
| company_profile	| A brief company description. |
| description	| The details description of the job ad. |
| requirements	| Enlisted requirements for the job opening. | 
| benefits	| Enlisted offered benefits by the employer. |
| telecommuting	| True for telecommuting positions. |
| hascompanylogo	| True if company logo is present. |
| has_questions	| True if screening questions are present. |
| employment_type	| Full-type, Part-time, Contract, etc. |
| required_experience	| Executive, Entry level, Intern, etc. |
| required_education	| Doctorate, Master’s Degree, Bachelor, etc. |
| industry	| Automotive, IT, Health care, Real estate, etc. |
| function	| Consulting, Engineering, Research, Sales, etc. |
| fraudulent	| The target and binary classification attribute. |

In [4]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
# import tensorflow as tf
# import keras

In [5]:
# read in the file
df = pd.read_csv("fake_job_postings.csv")

In [6]:
# glance at the dataset
df.head()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [7]:
# take a look at the column names
df.columns

Index(['job_id', 'title', 'location', 'department', 'salary_range',
       'company_profile', 'description', 'requirements', 'benefits',
       'telecommuting', 'has_company_logo', 'has_questions', 'employment_type',
       'required_experience', 'required_education', 'industry', 'function',
       'fraudulent'],
      dtype='object')

In [8]:
# there is a lot of missing data across the features and most of the features are categorical
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17880 entries, 0 to 17879
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   job_id               17880 non-null  int64 
 1   title                17880 non-null  object
 2   location             17534 non-null  object
 3   department           6333 non-null   object
 4   salary_range         2868 non-null   object
 5   company_profile      14572 non-null  object
 6   description          17879 non-null  object
 7   requirements         15185 non-null  object
 8   benefits             10670 non-null  object
 9   telecommuting        17880 non-null  int64 
 10  has_company_logo     17880 non-null  int64 
 11  has_questions        17880 non-null  int64 
 12  employment_type      14409 non-null  object
 13  required_experience  10830 non-null  object
 14  required_education   9775 non-null   object
 15  industry             12977 non-null  object
 16  func

In [9]:
# gauge the shape of the dataset
df.shape

(17880, 18)

In [10]:
# the breakdown between real and fradulent job postings is not even (17,014 real and 866 fake, respectively)

df.iloc[:,-1].value_counts()

0    17014
1      866
Name: fraudulent, dtype: int64

## Feature Engineering

While there are 18 features to choose from, I do not believe that they are all important for my classification problem. As we can see from the below subset of unique locations, they are not all consistently formatted. We can try grabbing the first two characters to create a country column. This way we can see the proportion of fraudulent job postings in different countries.

We also see that most of the columns have a lot of null values. We can likely disregard these features for our classification because they probably do not matter much. The most important feature is the description which we can use natural language processing (NLP) on to find patterns between fake job postings.

In [11]:
df['location'].unique()

array(['US, NY, New York', 'NZ, , Auckland', 'US, IA, Wever', ...,
       'US, CA, los Angeles', 'CA, , Ottawa', 'GB, WSX, Chichester'],
      dtype=object)

In [12]:
# countries mapped to country codes for matching later

country_dict = {'Afghanistan': 'AF',
 'Albania': 'AL',
 'Algeria': 'DZ',
 'American Samoa': 'AS',
 'Andorra': 'AD',
 'Angola': 'AO',
 'Anguilla': 'AI',
 'Antarctica': 'AQ',
 'Antigua and Barbuda': 'AG',
 'Argentina': 'AR',
 'Armenia': 'AM',
 'Aruba': 'AW',
 'Australia': 'AU',
 'Austria': 'AT',
 'Azerbaijan': 'AZ',
 'Bahamas': 'BS',
 'Bahrain': 'BH',
 'Bangladesh': 'BD',
 'Barbados': 'BB',
 'Belarus': 'BY',
 'Belgium': 'BE',
 'Belize': 'BZ',
 'Benin': 'BJ',
 'Bermuda': 'BM',
 'Bhutan': 'BT',
 'Bolivia, Plurinational State of': 'BO',
 'Bonaire, Sint Eustatius and Saba': 'BQ',
 'Bosnia and Herzegovina': 'BA',
 'Botswana': 'BW',
 'Bouvet Island': 'BV',
 'Brazil': 'BR',
 'British Indian Ocean Territory': 'IO',
 'Brunei Darussalam': 'BN',
 'Bulgaria': 'BG',
 'Burkina Faso': 'BF',
 'Burundi': 'BI',
 'Cambodia': 'KH',
 'Cameroon': 'CM',
 'Canada': 'CA',
 'Cape Verde': 'CV',
 'Cayman Islands': 'KY',
 'Central African Republic': 'CF',
 'Chad': 'TD',
 'Chile': 'CL',
 'China': 'CN',
 'Christmas Island': 'CX',
 'Cocos (Keeling) Islands': 'CC',
 'Colombia': 'CO',
 'Comoros': 'KM',
 'Congo': 'CG',
 'Congo, the Democratic Republic of the': 'CD',
 'Cook Islands': 'CK',
 'Costa Rica': 'CR',
 'Croatia': 'HR',
 'Cuba': 'CU',
 'Curaçao': 'CW',
 'Cyprus': 'CY',
 'Czech Republic': 'CZ',
 "Côte d'Ivoire": 'CI',
 'Denmark': 'DK',
 'Djibouti': 'DJ',
 'Dominica': 'DM',
 'Dominican Republic': 'DO',
 'Ecuador': 'EC',
 'Egypt': 'EG',
 'El Salvador': 'SV',
 'Equatorial Guinea': 'GQ',
 'Eritrea': 'ER',
 'Estonia': 'EE',
 'Ethiopia': 'ET',
 'Falkland Islands (Malvinas)': 'FK',
 'Faroe Islands': 'FO',
 'Fiji': 'FJ',
 'Finland': 'FI',
 'France': 'FR',
 'French Guiana': 'GF',
 'French Polynesia': 'PF',
 'French Southern Territories': 'TF',
 'Gabon': 'GA',
 'Gambia': 'GM',
 'Georgia': 'GE',
 'Germany': 'DE',
 'Ghana': 'GH',
 'Gibraltar': 'GI',
 'Greece': 'GR',
 'Greenland': 'GL',
 'Grenada': 'GD',
 'Guadeloupe': 'GP',
 'Guam': 'GU',
 'Guatemala': 'GT',
 'Guernsey': 'GG',
 'Guinea': 'GN',
 'Guinea-Bissau': 'GW',
 'Guyana': 'GY',
 'Haiti': 'HT',
 'Heard Island and McDonald Islands': 'HM',
 'Holy See (Vatican City State)': 'VA',
 'Honduras': 'HN',
 'Hong Kong': 'HK',
 'Hungary': 'HU',
 'Iceland': 'IS',
 'India': 'IN',
 'Indonesia': 'ID',
 'Iran, Islamic Republic of': 'IR',
 'Iraq': 'IQ',
 'Ireland': 'IE',
 'Isle of Man': 'IM',
 'Israel': 'IL',
 'Italy': 'IT',
 'Jamaica': 'JM',
 'Japan': 'JP',
 'Jersey': 'JE',
 'Jordan': 'JO',
 'Kazakhstan': 'KZ',
 'Kenya': 'KE',
 'Kiribati': 'KI',
 "Korea, Democratic People's Republic of": 'KP',
 'Korea, Republic of': 'KR',
 'Kuwait': 'KW',
 'Kyrgyzstan': 'KG',
 "Lao People's Democratic Republic": 'LA',
 'Latvia': 'LV',
 'Lebanon': 'LB',
 'Lesotho': 'LS',
 'Liberia': 'LR',
 'Libya': 'LY',
 'Liechtenstein': 'LI',
 'Lithuania': 'LT',
 'Luxembourg': 'LU',
 'Macao': 'MO',
 'Macedonia, the former Yugoslav Republic of': 'MK',
 'Madagascar': 'MG',
 'Malawi': 'MW',
 'Malaysia': 'MY',
 'Maldives': 'MV',
 'Mali': 'ML',
 'Malta': 'MT',
 'Marshall Islands': 'MH',
 'Martinique': 'MQ',
 'Mauritania': 'MR',
 'Mauritius': 'MU',
 'Mayotte': 'YT',
 'Mexico': 'MX',
 'Micronesia, Federated States of': 'FM',
 'Moldova, Republic of': 'MD',
 'Monaco': 'MC',
 'Mongolia': 'MN',
 'Montenegro': 'ME',
 'Montserrat': 'MS',
 'Morocco': 'MA',
 'Mozambique': 'MZ',
 'Myanmar': 'MM',
 'Namibia': 'NA',
 'Nauru': 'NR',
 'Nepal': 'NP',
 'Netherlands': 'NL',
 'New Caledonia': 'NC',
 'New Zealand': 'NZ',
 'Nicaragua': 'NI',
 'Niger': 'NE',
 'Nigeria': 'NG',
 'Niue': 'NU',
 'Norfolk Island': 'NF',
 'Northern Mariana Islands': 'MP',
 'Norway': 'NO',
 'Oman': 'OM',
 'Pakistan': 'PK',
 'Palau': 'PW',
 'Palestine, State of': 'PS',
 'Panama': 'PA',
 'Papua New Guinea': 'PG',
 'Paraguay': 'PY',
 'Peru': 'PE',
 'Philippines': 'PH',
 'Pitcairn': 'PN',
 'Poland': 'PL',
 'Portugal': 'PT',
 'Puerto Rico': 'PR',
 'Qatar': 'QA',
 'Romania': 'RO',
 'Russian Federation': 'RU',
 'Rwanda': 'RW',
 'Réunion': 'RE',
 'Saint Barthélemy': 'BL',
 'Saint Helena, Ascension and Tristan da Cunha': 'SH',
 'Saint Kitts and Nevis': 'KN',
 'Saint Lucia': 'LC',
 'Saint Martin (French part)': 'MF',
 'Saint Pierre and Miquelon': 'PM',
 'Saint Vincent and the Grenadines': 'VC',
 'Samoa': 'WS',
 'San Marino': 'SM',
 'Sao Tome and Principe': 'ST',
 'Saudi Arabia': 'SA',
 'Senegal': 'SN',
 'Serbia': 'RS',
 'Seychelles': 'SC',
 'Sierra Leone': 'SL',
 'Singapore': 'SG',
 'Sint Maarten (Dutch part)': 'SX',
 'Slovakia': 'SK',
 'Slovenia': 'SI',
 'Solomon Islands': 'SB',
 'Somalia': 'SO',
 'South Africa': 'ZA',
 'South Georgia and the South Sandwich Islands': 'GS',
 'South Sudan': 'SS',
 'Spain': 'ES',
 'Sri Lanka': 'LK',
 'Sudan': 'SD',
 'Suriname': 'SR',
 'Svalbard and Jan Mayen': 'SJ',
 'Swaziland': 'SZ',
 'Sweden': 'SE',
 'Switzerland': 'CH',
 'Syrian Arab Republic': 'SY',
 'Taiwan, Province of China': 'TW',
 'Tajikistan': 'TJ',
 'Tanzania, United Republic of': 'TZ',
 'Thailand': 'TH',
 'Timor-Leste': 'TL',
 'Togo': 'TG',
 'Tokelau': 'TK',
 'Tonga': 'TO',
 'Trinidad and Tobago': 'TT',
 'Tunisia': 'TN',
 'Turkey': 'TR',
 'Turkmenistan': 'TM',
 'Turks and Caicos Islands': 'TC',
 'Tuvalu': 'TV',
 'Uganda': 'UG',
 'Ukraine': 'UA',
 'United Arab Emirates': 'AE',
 'United Kingdom': 'GB',
 'United States': 'US',
 'United States Minor Outlying Islands': 'UM',
 'Uruguay': 'UY',
 'Uzbekistan': 'UZ',
 'Vanuatu': 'VU',
 'Venezuela, Bolivarian Republic of': 'VE',
 'Viet Nam': 'VN',
 'Virgin Islands, British': 'VG',
 'Virgin Islands, U.S.': 'VI',
 'Wallis and Futuna': 'WF',
 'Western Sahara': 'EH',
 'Yemen': 'YE',
 'Zambia': 'ZM',
 'Zimbabwe': 'ZW',
 'Åland Islands': 'AX'}

In [13]:
# function to match country codes to country names
def match_code_to_country(code):
    for key, value in country_dict.items():
        if code in value:
            return key

In [14]:
# create a new country column using the first two characters of the location column and put it next to location

df["location"] = df["location"].apply(str) # ensure that location is a string

location_index = df.columns.get_loc('location') # location column index

df['country_code'] = df['location'].str[:2] # grab first two characters from the string

df.insert((location_index+1),'country', df['country_code'].apply(match_code_to_country)) # insert country next to location

df['country']

0        United States
1          New Zealand
2        United States
3        United States
4        United States
             ...      
17875           Canada
17876    United States
17877    United States
17878          Nigeria
17879      New Zealand
Name: country, Length: 17880, dtype: object

In [15]:
# remove the country code column as we don't need it

df.pop('country_code')

0        US
1        NZ
2        US
3        US
4        US
         ..
17875    CA
17876    US
17877    US
17878    NG
17879    NZ
Name: country_code, Length: 17880, dtype: object

In [16]:
# look at the unique countries
df['country'].unique()

array(['United States', 'New Zealand', 'Germany', 'United Kingdom',
       'Australia', 'Singapore', 'Israel', 'United Arab Emirates',
       'Canada', 'India', 'Egypt', 'Poland', 'Greece', None, 'Pakistan',
       'Belgium', 'Brazil', 'Saudi Arabia', 'Denmark',
       'Russian Federation', 'South Africa', 'Cyprus', 'Hong Kong',
       'Turkey', 'Ireland', 'Lithuania', 'Japan', 'Netherlands',
       'Austria', 'Korea, Republic of', 'France', 'Estonia', 'Thailand',
       'Panama', 'Kenya', 'Mauritius', 'Mexico', 'Romania', 'Malaysia',
       'Finland', 'China', 'Spain', 'Sweden', 'Chile', 'Ukraine', 'Qatar',
       'Italy', 'Latvia', 'Iraq', 'Bulgaria', 'Philippines',
       'Czech Republic', 'Virgin Islands, U.S.', 'Malta', 'Hungary',
       'Bangladesh', 'Kuwait', 'Luxembourg', 'Nigeria', 'Serbia',
       'Belarus', 'Viet Nam', 'Indonesia', 'Zambia', 'Norway', 'Bahrain',
       'Uganda', 'Switzerland', 'Trinidad and Tobago', 'Sudan',
       'Slovakia', 'Argentina', 'Taiwan, Province 

In [17]:
# there are still 346 missing country values
df['country'].isna().sum()

346

In [18]:
# replace the none values with Unknown so it makes more sense
df['country'].fillna(value="Unknown", inplace=True)
df['country'].unique()

array(['United States', 'New Zealand', 'Germany', 'United Kingdom',
       'Australia', 'Singapore', 'Israel', 'United Arab Emirates',
       'Canada', 'India', 'Egypt', 'Poland', 'Greece', 'Unknown',
       'Pakistan', 'Belgium', 'Brazil', 'Saudi Arabia', 'Denmark',
       'Russian Federation', 'South Africa', 'Cyprus', 'Hong Kong',
       'Turkey', 'Ireland', 'Lithuania', 'Japan', 'Netherlands',
       'Austria', 'Korea, Republic of', 'France', 'Estonia', 'Thailand',
       'Panama', 'Kenya', 'Mauritius', 'Mexico', 'Romania', 'Malaysia',
       'Finland', 'China', 'Spain', 'Sweden', 'Chile', 'Ukraine', 'Qatar',
       'Italy', 'Latvia', 'Iraq', 'Bulgaria', 'Philippines',
       'Czech Republic', 'Virgin Islands, U.S.', 'Malta', 'Hungary',
       'Bangladesh', 'Kuwait', 'Luxembourg', 'Nigeria', 'Serbia',
       'Belarus', 'Viet Nam', 'Indonesia', 'Zambia', 'Norway', 'Bahrain',
       'Uganda', 'Switzerland', 'Trinidad and Tobago', 'Sudan',
       'Slovakia', 'Argentina', 'Taiwan, Prov

In [19]:
# check how many null values each column has
df.isnull().sum()

job_id                     0
title                      0
location                   0
country                    0
department             11547
salary_range           15012
company_profile         3308
description                1
requirements            2695
benefits                7210
telecommuting              0
has_company_logo           0
has_questions              0
employment_type         3471
required_experience     7050
required_education      8105
industry                4903
function                6455
fraudulent                 0
dtype: int64

In [20]:
# find the columns without null values and add them to a list
no_null_columns = df.columns[df.notna().all()].tolist()
no_null_columns

['job_id',
 'title',
 'location',
 'country',
 'telecommuting',
 'has_company_logo',
 'has_questions',
 'fraudulent']

In [21]:
# let us remove location since we have country already and add description because it is important
important_features = no_null_columns
important_features[no_null_columns.index('location')] = 'description'

In [22]:
# check to make sure it worked correctly
important_features

['job_id',
 'title',
 'description',
 'country',
 'telecommuting',
 'has_company_logo',
 'has_questions',
 'fraudulent']

In [23]:
df2 = df[important_features].copy()

In [24]:
df2

,job_id,title,description,country,telecommuting,has_company_logo,has_questions,fraudulent
0,1,Marketing Intern,"Food52, a fast-growing, James Beard Award-winn...",United States,0,1,0,0
1,2,Customer Service - Cloud Video Production,Organised - Focused - Vibrant - Awesome!Do you...,New Zealand,0,1,0,0
2,3,Commissioning Machinery Assistant (CMA),"Our client, located in Houston, is actively se...",United States,0,1,0,0
3,4,Account Executive - Washington DC,THE COMPANY: ESRI – Environmental Systems Rese...,United States,0,1,0,0
4,5,Bill Review Manager,JOB TITLE: Itemization Review ManagerLOCATION:...,United States,0,1,1,0
...,...,...,...,...,...,...,...,...
17875,17876,Account Director - Distribution,Just in case this is the first time you’ve vis...,Canada,0,1,1,0
17876,17877,Payroll Accountant,The Payroll Accountant will focus primarily on...,United States,0,1,1,0
17877,17878,Project Cost Control Staff Engineer - Cost Con...,Experienced Project Cost Control Staff Enginee...,United States,0,0,0,0
17878,17879,Graphic Designer,Nemsia Studios is looking for an experienced v...,Nigeria,0,0,1,0


In [25]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17880 entries, 0 to 17879
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   job_id            17880 non-null  int64 
 1   title             17880 non-null  object
 2   description       17879 non-null  object
 3   country           17880 non-null  object
 4   telecommuting     17880 non-null  int64 
 5   has_company_logo  17880 non-null  int64 
 6   has_questions     17880 non-null  int64 
 7   fraudulent        17880 non-null  int64 
dtypes: int64(5), object(3)
memory usage: 1.1+ MB


In [26]:
# since there is one null value in description, let us drop it
df2.dropna(inplace=True)

In [27]:
df2

,job_id,title,description,country,telecommuting,has_company_logo,has_questions,fraudulent
0,1,Marketing Intern,"Food52, a fast-growing, James Beard Award-winn...",United States,0,1,0,0
1,2,Customer Service - Cloud Video Production,Organised - Focused - Vibrant - Awesome!Do you...,New Zealand,0,1,0,0
2,3,Commissioning Machinery Assistant (CMA),"Our client, located in Houston, is actively se...",United States,0,1,0,0
3,4,Account Executive - Washington DC,THE COMPANY: ESRI – Environmental Systems Rese...,United States,0,1,0,0
4,5,Bill Review Manager,JOB TITLE: Itemization Review ManagerLOCATION:...,United States,0,1,1,0
...,...,...,...,...,...,...,...,...
17875,17876,Account Director - Distribution,Just in case this is the first time you’ve vis...,Canada,0,1,1,0
17876,17877,Payroll Accountant,The Payroll Accountant will focus primarily on...,United States,0,1,1,0
17877,17878,Project Cost Control Staff Engineer - Cost Con...,Experienced Project Cost Control Staff Enginee...,United States,0,0,0,0
17878,17879,Graphic Designer,Nemsia Studios is looking for an experienced v...,Nigeria,0,0,1,0


In [28]:
country_comp = pd.crosstab(df2["country"],df2["fraudulent"])

In [29]:
country_comp["Total"] = country_comp[1] + country_comp[0]
country_comp["Percentage"] = round((country_comp[1] / country_comp["Total"])*100,2)

In [30]:
country_comp = country_comp[[1, 0, "Total", "Percentage"]]
country_comp.rename(columns={1:"Fake", 0:"Real"},inplace=True)

An overwhelming amount of total job postings in the dataset are from the United States. We see below that a few countries had more fake postings than real ones. Overall, given how skewed the dataset is towards the US, we may want to remove country as a feature when we build our classification model.

In [31]:
# sort the values by highest fake postings percentage
country_comp.sort_values(by=["Percentage"],ascending=False, inplace=True)
country_comp.head(20)

fraudulent,Fake,Real,Total,Percentage
country,,,,
Malaysia,12,9,21,57.14
Bahrain,5,4,9,55.56
"Taiwan, Province of China",2,2,4,50.00
Qatar,6,15,21,28.57
Australia,40,174,214,18.69
Indonesia,1,12,13,7.69
United States,730,9926,10656,6.85
Saudi Arabia,1,14,15,6.67
Unknown,19,327,346,5.49


In [32]:
countries = country_comp.index

In [33]:
countries

Index(['Malaysia', 'Bahrain', 'Taiwan, Province of China', 'Qatar',
       'Australia', 'Indonesia', 'United States', 'Saudi Arabia', 'Unknown',
       'Pakistan', 'Brazil', 'Poland', 'Canada', 'South Africa', 'Egypt',
       'United Arab Emirates', 'Spain', 'India', 'Estonia', 'United Kingdom',
       'Philippines', 'Panama', 'Peru', 'Nigeria', 'Nicaragua', 'Portugal',
       'New Zealand', 'Romania', 'Russian Federation', 'Netherlands', 'Norway',
       'Albania', 'Serbia', 'Singapore', 'Virgin Islands, U.S.', 'Viet Nam',
       'Ukraine', 'Uganda', 'Turkey', 'Tunisia', 'Trinidad and Tobago',
       'Thailand', 'Switzerland', 'Sweden', 'Sudan', 'Sri Lanka', 'Mexico',
       'Slovenia', 'Slovakia', 'Morocco', 'Lithuania', 'Mauritius', 'Malta',
       'Finland', 'El Salvador', 'Denmark', 'Czech Republic', 'Cyprus',
       'Croatia', 'Colombia', 'China', 'Chile', 'Cameroon', 'Cambodia',
       'Bulgaria', 'Belgium', 'Belarus', 'Bangladesh', 'Austria', 'Armenia',
       'France', 'German

Out of curiousity, I mapped the breakout of different job postings across the globe. It is a cool visual but not useful for our model.

In [34]:
import geocoder
import folium

In [35]:
lat = []
long = []
for c in countries:
    lat.append(geocoder.arcgis(c).json['lat'])
    long.append(geocoder.arcgis(c).json['lng'])
country_comp["lat"] = lat
country_comp["long"] = long
country_comp

fraudulent,Fake,Real,Total,Percentage,lat,long
country,,,,,,
Malaysia,12,9,21,57.14,2.500000,112.500000
Bahrain,5,4,9,55.56,26.051859,50.564427
"Taiwan, Province of China",2,2,4,50.00,32.328120,118.763840
Qatar,6,15,21,28.57,25.285445,51.193148
Australia,40,174,214,18.69,-25.709932,134.484031
...,...,...,...,...,...,...
Iraq,0,10,10,0.00,33.038975,43.777172
Iceland,0,2,2,0.00,64.986957,-18.582382
Hungary,0,14,14,0.00,47.167089,19.424532


In [36]:
ad_map = folium.Map(location=[20,0], tiles="OpenStreetMap", zoom_start=2)

In [37]:
for i in range(0,len(country_comp)):
    html=f"""
        <h1> {country_comp.index[i]}</h1>
        <ul>
            <li>Fake Ads: {country_comp.iloc[i]['Fake']}</li>
            <li>Real Ads: {country_comp.iloc[i]['Real']}</li>
            <li>Total Ads: {country_comp.iloc[i]['Total']}</li>
            <li><strong>% Fake Ads: {country_comp.iloc[i]['Percentage']}</strong></li>
        </ul>
        """
    iframe = folium.IFrame(html=html, width=200, height=200)
    popup = folium.Popup(iframe, max_width=2650)
    folium.Marker(
        location=[country_comp.iloc[i]['lat'], country_comp.iloc[i]['long']],
        popup=popup,
        icon=folium.DivIcon(html=f"""
            <div><svg>
                <circle cx="50" cy="50" r="40" fill="#69b3a2" opacity=".4"/>
                <rect x="35", y="35" width="30" height="30", fill="red", opacity=".3" 
            </svg></div>""")
    ).add_to(ad_map)

In [38]:
ad_map

## Preprocessing

Now that we see that country is probably not a great predictor of our class based on the above analysis, we can begin preprocessing the rest of the data. To prepare the data for classification, we need to analyze the description feature which is likely the biggest determinant of our class.

In [39]:
df2

,job_id,title,description,country,telecommuting,has_company_logo,has_questions,fraudulent
0,1,Marketing Intern,"Food52, a fast-growing, James Beard Award-winn...",United States,0,1,0,0
1,2,Customer Service - Cloud Video Production,Organised - Focused - Vibrant - Awesome!Do you...,New Zealand,0,1,0,0
2,3,Commissioning Machinery Assistant (CMA),"Our client, located in Houston, is actively se...",United States,0,1,0,0
3,4,Account Executive - Washington DC,THE COMPANY: ESRI – Environmental Systems Rese...,United States,0,1,0,0
4,5,Bill Review Manager,JOB TITLE: Itemization Review ManagerLOCATION:...,United States,0,1,1,0
...,...,...,...,...,...,...,...,...
17875,17876,Account Director - Distribution,Just in case this is the first time you’ve vis...,Canada,0,1,1,0
17876,17877,Payroll Accountant,The Payroll Accountant will focus primarily on...,United States,0,1,1,0
17877,17878,Project Cost Control Staff Engineer - Cost Con...,Experienced Project Cost Control Staff Enginee...,United States,0,0,0,0
17878,17879,Graphic Designer,Nemsia Studios is looking for an experienced v...,Nigeria,0,0,1,0


There are a few rows where the description is essentially missing. We can safely drop those without it affecting our model.

In [40]:
# check to see if there are really short descriptions that are not useful
df2[df2['description'].apply(len) < 9]

,job_id,title,description,country,telecommuting,has_company_logo,has_questions,fraudulent
3030,3031,mobile apps for Android/iOS developer,#NAME?,Viet Nam,0,0,0,0
3951,3952,Senior Java Developer with Hadoop Exp,#NAME?,United States,0,1,0,0
4184,4185,Senior Architect / Technician,#NAME?,Philippines,0,1,0,0
7612,7613,Mobile Marketing Specialist,#NAME?,Hong Kong,0,1,1,0
11893,11894,Junior Specialist - Seed Production and Harvest,#NAME?,Philippines,0,0,0,0
12189,12190,Researcher - Nutrient and Crop Management,#NAME?,Philippines,0,0,0,0
13529,13530,Full-time Web Developer,#NAME?,Philippines,0,1,0,0
16690,16691,Marketing Lead,#NAME?,Unknown,0,1,0,0


In [41]:
df2.drop(df2[df2['description'].apply(len) < 9].index, inplace=True)

In [42]:
# check if they were all deleted successfully
df2['description'].str.contains('#NAME?').sum()

0

Now that we determined that the country is not going to be important for our model, we can drop it from our dataframe. We can also drop job_id which is just an arbitrary sequential ID assigned to each job ad. It has no practical use for our model.

In [43]:
df2.pop("country")
df2.pop("job_id")

0            1
1            2
2            3
3            4
4            5
         ...  
17875    17876
17876    17877
17877    17878
17878    17879
17879    17880
Name: job_id, Length: 17871, dtype: int64

Let us now experiment with the natural language toolkit (NLTK) library to transform the description column. This will allow us to remove unnecessary words and hone it on those that can differentiate real job ads from fake ones. We will first test on one description value until we find an optimal method. After that, we will apply it to the whole description column.

In [44]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [45]:
dummy_text = df2["description"][0]
dummy_text

'Food52, a fast-growing, James Beard Award-winning online food community and crowd-sourced and curated recipe hub, is currently interviewing full- and part-time unpaid interns to work in a small team of editors, executives, and developers in its New York City headquarters.Reproducing and/or repackaging existing Food52 content for a number of partner sites, such as Huffington Post, Yahoo, Buzzfeed, and more in their various content management systemsResearching blogs and websites for the Provisions by Food52 Affiliate ProgramAssisting in day-to-day affiliate program support, such as screening affiliates and assisting in any affiliate inquiriesSupporting with PR &amp; Events when neededHelping with office administrative work, such as filing, mailing, and preparing for meetingsWorking with developers to document bugs and suggest improvements to the siteSupporting the marketing and executive staff'

In [46]:
nltk.download('omw-1.4')
nltk.download("stopwords")

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\stanl\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\stanl\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [47]:
# create a function to transform the text by removing punctuation, stop words, and lemmatizing the words

def transform_text(text):
    # first we can remove the punctuation
    tokenizer = RegexpTokenizer(r'\w+') #search for alphanumeric characters
    text = tokenizer.tokenize(text.casefold())

    # next, remove the stop words
    stop_words = set(stopwords.words("english"))
    filtered_list = [word for word in text if word not in stop_words]


    # finally, lemmatize the text to derive the core meaning of each word
    lemmatizer = WordNetLemmatizer() #reduces words to their core meaning
    text = [lemmatizer.lemmatize(w) for w in filtered_list]
    text = " ".join(text)
    
    return text

In [48]:
dummy = transform_text(dummy_text)

In [49]:
dummy

'food52 fast growing james beard award winning online food community crowd sourced curated recipe hub currently interviewing full part time unpaid intern work small team editor executive developer new york city headquarters reproducing repackaging existing food52 content number partner site huffington post yahoo buzzfeed various content management systemsresearching blog website provision food52 affiliate programassisting day day affiliate program support screening affiliate assisting affiliate inquiriessupporting pr amp event neededhelping office administrative work filing mailing preparing meetingsworking developer document bug suggest improvement sitesupporting marketing executive staff'

### Note
It looks like our function works correctly. It was able to remove the punctuation, remove common words that do not add any value (stop words), and lemmatize the words. We can now apply the function to the whole description column.

In [50]:
# helps improve the speed of the apply function
import swifter

In [51]:
df2['description'] = df2['description'].swifter.apply(transform_text)

Pandas Apply:   0%|          | 0/17871 [00:00<?, ?it/s]

In [52]:
# check to ensure that it worked

df2.head()

,title,description,telecommuting,has_company_logo,has_questions,fraudulent
0,Marketing Intern,food52 fast growing james beard award winning ...,0,1,0,0
1,Customer Service - Cloud Video Production,organised focused vibrant awesome passion cust...,0,1,0,0
2,Commissioning Machinery Assistant (CMA),client located houston actively seeking experi...,0,1,0,0
3,Account Executive - Washington DC,company esri environmental system research ins...,0,1,0,0
4,Bill Review Manager,job title itemization review managerlocation f...,0,1,1,0


In [53]:
df2["description"][0]

'food52 fast growing james beard award winning online food community crowd sourced curated recipe hub currently interviewing full part time unpaid intern work small team editor executive developer new york city headquarters reproducing repackaging existing food52 content number partner site huffington post yahoo buzzfeed various content management systemsresearching blog website provision food52 affiliate programassisting day day affiliate program support screening affiliate assisting affiliate inquiriessupporting pr amp event neededhelping office administrative work filing mailing preparing meetingsworking developer document bug suggest improvement sitesupporting marketing executive staff'

In [54]:
# split the data into X and Y
X = df2.iloc[:,0:-1]
y = df2.iloc[:,-1]

In [55]:
# one-hot encode the title column
X = pd.get_dummies(X, columns=["title"])

In [56]:
# check to see if it worked

X

,description,telecommuting,has_company_logo,has_questions,title_ Electrician,title_ Environmental Technician I,title_ Piping Material Engineer,"title_ Discipline Manager Civil, Structural, Marine, Architectural",title_ FEA Senior engineer,title_ Manager of Project Management Organization - Engineering,...,title_talented Senior Python developer,title_teacher,"title_vacancies,nannies,cooks,drivers and housekeepers needed",title_warehouse Clerical/Superusers,title_warehouse first line manager/ Shift Manager,title_warehouse opperative,title_web designer,title_web project manager,title_work from home,title_~ LM Structures - Expression of Interest for Future Consideration
0,food52 fast growing james beard award winning ...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,organised focused vibrant awesome passion cust...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,client located houston actively seeking experi...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,company esri environmental system research ins...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,job title itemization review managerlocation f...,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17875,case first time visited website vend award win...,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17876,payroll accountant focus primarily payroll fun...,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17877,experienced project cost control staff enginee...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17878,nemsia studio looking experienced visual graph...,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [57]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [58]:
# split the data into training, testing, and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42) # 0.25 x 0.8 = 0.2

In [59]:
# check that the splitting worked correctly
print("X Training Shape:", X_train.shape)
print("Y Training Shape:", y_train.shape)
print("X Validation Shape:", X_val.shape)
print("Y Validation Shape:", y_val.shape)
print("X Test Shape:", X_test.shape)
print("Y Test Shape:", y_test.shape)

X Training Shape: (10722, 11228)
Y Training Shape: (10722,)
X Validation Shape: (3574, 11228)
Y Validation Shape: (3574,)
X Test Shape: (3575, 11228)
Y Test Shape: (3575,)


In [60]:
# create a TfidVectorizer object to convert the individual words in the description into a matrix
tfid = TfidfVectorizer(min_df=2) # exclude words that only appear once
X_train2 = tfid.fit_transform(X_train["description"].values)
X_val2 = tfid.transform(X_val["description"].values)
X_test2 = tfid.transform(X_test["description"].values)

In [61]:
X_train2.shape

(10722, 18977)

In [62]:
# create a DF to map the feature names to the matrix so we can append it to the original training DF
training = pd.DataFrame(X_train2.toarray(), columns=tfid.get_feature_names_out())
training

,00,000,0001pt,000bonus,00am,00pm,00pmlocation,00salary,01,01858965,...,한국에,한국에서,한국의,허니스크린,허니스크린은,혁명적으로,현재,화면이란,확장을,활용해
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10717,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10718,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10719,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10720,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [63]:
# remove the description column since we will add vectorized versions of it
X_train.pop("description")
X_val.pop("description")
X_test.pop("description")

4869     love customer fact ask 2 million best thing pl...
2726     cook responsible planning preparation meal cli...
8869     client one leading saas based provider clinica...
9628     develop execute recruiting plan network indust...
12047    design core backend frontend software componen...
                               ...                        
11391    work love earn u best bet atlanta looking dedi...
5957     customer service associate based san francisco...
16898    duty responsibility 1 marketing representative...
2112     transferwho co founded skype first employee ba...
4284     hrdownloads canada leading provider human reso...
Name: description, Length: 3575, dtype: object

In [64]:
X_train.reset_index(inplace=True, drop=True)
training.reset_index(inplace=True, drop=True)

In [65]:
print("X_train Index:", X_train.index, "Shape:", X_train.shape)
print("Training Index:", training.index, "Shape:", training.shape)

X_train Index: RangeIndex(start=0, stop=10722, step=1) Shape: (10722, 11227)
Training Index: RangeIndex(start=0, stop=10722, step=1) Shape: (10722, 18977)


In [66]:
X_train_vec = pd.concat([X_train, training], axis=1)
X_train_vec

,telecommuting,has_company_logo,has_questions,title_ Electrician,title_ Environmental Technician I,title_ Piping Material Engineer,"title_ Discipline Manager Civil, Structural, Marine, Architectural",title_ FEA Senior engineer,title_ Manager of Project Management Organization - Engineering,title_ Mechanical Engineering Manager,...,한국에,한국에서,한국의,허니스크린,허니스크린은,혁명적으로,현재,화면이란,확장을,활용해
0,0,1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10717,0,1,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10718,0,0,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10719,0,1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10720,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [67]:
# map validation matrix to feature names
validation = pd.DataFrame(X_val2.toarray(), columns=tfid.get_feature_names_out())
validation

,00,000,0001pt,000bonus,00am,00pm,00pmlocation,00salary,01,01858965,...,한국에,한국에서,한국의,허니스크린,허니스크린은,혁명적으로,현재,화면이란,확장을,활용해
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3569,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3570,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3571,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3572,0.0,0.048456,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [68]:
X_val.reset_index(inplace=True, drop=True)
validation.reset_index(inplace=True, drop=True)

print("X_val Index:", X_val.index, "Shape:", X_val.shape)
print("Validation Index:", validation.index, "Shape:", validation.shape)

X_val Index: RangeIndex(start=0, stop=3574, step=1) Shape: (3574, 11227)
Validation Index: RangeIndex(start=0, stop=3574, step=1) Shape: (3574, 18977)


In [69]:
X_val_vec = pd.concat([X_val, validation], axis=1)
X_val_vec

,telecommuting,has_company_logo,has_questions,title_ Electrician,title_ Environmental Technician I,title_ Piping Material Engineer,"title_ Discipline Manager Civil, Structural, Marine, Architectural",title_ FEA Senior engineer,title_ Manager of Project Management Organization - Engineering,title_ Mechanical Engineering Manager,...,한국에,한국에서,한국의,허니스크린,허니스크린은,혁명적으로,현재,화면이란,확장을,활용해
0,0,1,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,1,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,1,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3569,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3570,0,1,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3571,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3572,0,1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [70]:
# map testing matrix to feature names
testing = pd.DataFrame(X_test2.toarray(), columns=tfid.get_feature_names_out())
testing

,00,000,0001pt,000bonus,00am,00pm,00pmlocation,00salary,01,01858965,...,한국에,한국에서,한국의,허니스크린,허니스크린은,혁명적으로,현재,화면이란,확장을,활용해
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3570,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3572,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3573,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [71]:
X_test.reset_index(inplace=True, drop=True)
testing.reset_index(inplace=True, drop=True)

print("X_test Index:", X_test.index, "Shape:", X_test.shape)
print("Testing Index:", testing.index, "Shape:", testing.shape)

X_test Index: RangeIndex(start=0, stop=3575, step=1) Shape: (3575, 11227)
Testing Index: RangeIndex(start=0, stop=3575, step=1) Shape: (3575, 18977)


In [72]:
X_test_vec = pd.concat([X_test, testing], axis=1)
X_test_vec

,telecommuting,has_company_logo,has_questions,title_ Electrician,title_ Environmental Technician I,title_ Piping Material Engineer,"title_ Discipline Manager Civil, Structural, Marine, Architectural",title_ FEA Senior engineer,title_ Manager of Project Management Organization - Engineering,title_ Mechanical Engineering Manager,...,한국에,한국에서,한국의,허니스크린,허니스크린은,혁명적으로,현재,화면이란,확장을,활용해
0,0,1,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3570,0,1,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3571,0,1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3572,0,1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3573,0,1,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Now that we mapped the vectorized description terms to our original training, validation, and testing dataframes, we can begin building our models. We will start with a logistic regression.

## Logistic Regression

In [73]:
from sklearn.linear_model import LogisticRegression

In [74]:
# list for the regularization strength

strength = [1, 10]

To more efficiently find the right fit, solver, and C value for our logistic regression, we will leverage the below function. In a previous iteration, we tried five different C values (0.001, 0.01, 0.1, 1, and 10). The calculations were extremely slow and the results were consistent across models so we will only use 1 and 10 for this round. The kernel has trouble handling more.

In [75]:
# create a function to compute the different solvers
def run_reg(solver, penalty, C, random_state, max_iter, x_train, y_train, x_val, y_val):
    # create a logistic regression object and fit it
    model = LogisticRegression()
    model.solver = solver
    model.penalty = penalty
    model.C = C
    model.random_state = random_state
    model.max_iter = max_iter
    model.fit(x_train, y_train)
    
    # print the accuracy scores
    print(f"\033[1m{solver.title()} with C Value of {C}\033[0m")
    training_score = f"Training Accuracy Score:"
    validation_score = f"Validation Accuracy Score:"

    print(training_score, model.score(x_train, y_train)*100)
    print(validation_score, model.score(x_val, y_val)*100)
    print("----------------------------------------------")

### Sag Solver

We will start with the sag solver because it works well for large datasets. Our datasets have a very large number of features so certain solvers will struggle with it.

In [76]:
for c in strength:
    run_reg("sag","l2", c, 42, 3000, X_train_vec, y_train, X_val_vec, y_val)

Sag with C Value of 1
Training Accuracy Score: 97.50979294907667
Validation Accuracy Score: 96.36261891438164
----------------------------------------------
Sag with C Value of 10
Training Accuracy Score: 99.99067338183175
Validation Accuracy Score: 97.50979294907667
----------------------------------------------


### Lbfgs Solver

In [77]:
for c in strength:
    run_reg("lbfgs","l2", c, 42, 3000, X_train_vec, y_train, X_val_vec, y_val)

Lbfgs with C Value of 1
Training Accuracy Score: 97.50979294907667
Validation Accuracy Score: 96.36261891438164
----------------------------------------------
Lbfgs with C Value of 10
Training Accuracy Score: 99.99067338183175
Validation Accuracy Score: 97.50979294907667
----------------------------------------------


#### Liblinear L2

Liblinear is best suited for smaller datasets but we will give it a try to see how it does. We use both L2 and L1 regularization to compare the effects of the two penalties.

In [78]:
for c in strength:
    run_reg("liblinear","l2", c, 42, 3000, X_train_vec, y_train, X_val_vec, y_val)

Liblinear with C Value of 1
Training Accuracy Score: 97.50046633090841
Validation Accuracy Score: 96.36261891438164
----------------------------------------------
Liblinear with C Value of 10
Training Accuracy Score: 99.99067338183175
Validation Accuracy Score: 97.50979294907667
----------------------------------------------


### Liblinear L1

In [79]:
for c in strength:
    run_reg("liblinear","l1", c, 42, 3000, X_train_vec, y_train, X_val_vec, y_val)

Liblinear with C Value of 1
Training Accuracy Score: 97.38854691288938
Validation Accuracy Score: 96.61443760492445
----------------------------------------------
Liblinear with C Value of 10
Training Accuracy Score: 100.0
Validation Accuracy Score: 97.36989367655288
----------------------------------------------


### Saga L2

In [81]:
for c in strength:
    run_reg("saga","l2", c, 42, 3000, X_train_vec, y_train, X_val_vec, y_val)

Saga with C Value of 1
Training Accuracy Score: 97.50979294907667
Validation Accuracy Score: 96.36261891438164
----------------------------------------------
Saga with C Value of 10
Training Accuracy Score: 99.99067338183175
Validation Accuracy Score: 97.50979294907667
----------------------------------------------


### Saga L1

In [ ]:
for c in strength:
    run_reg("saga","l1", c, 42, 3000, X_train_vec, y_train, X_val_vec, y_val)

Saga with C Value of 1
Training Accuracy Score: 97.35124044021639
Validation Accuracy Score: 96.61443760492445
----------------------------------------------


### Observation

Interestingly, all of the solvers had roughly the same accuracy across the different C parameters. We will compare these results to those of other models to see if they make a difference. We know that our dataset is imbalanced so it is possible that the model is guessing the majority class every time and still getting a high accuracy.